In [1]:
from bs4 import BeautifulSoup
import requests as re
import pandas as pd
import numpy as np

In [6]:
%%time
col = ['year', 'month', 'url']
top_df = pd.DataFrame(columns=col)
#url = input("http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1")
url = "http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1"
path=url[:url.rfind('/')]
r = re.get(url)
data = r.text
year='None'
soup = BeautifulSoup(data, "lxml")
for element in soup.find_all(['a','strong']):
    if(element.string != None and element.string.strip().strip(':').strip().isdigit()):
        year=str(element.string.strip().strip(":").strip())
#         print('year changed to ' + year)
    if(element.get('href') != None and "./" in element.get('href')):
        month = (str(element.string).strip("."))
        monthurl = (path + element.get('href').strip('.'))
        row = pd.DataFrame([[year, month, monthurl]], columns=col)
        if(len(top_df)==0):
            top_df = row
        else:
            top_df = top_df.append(row, ignore_index=True) #why the hell doesn't df.append work inplace?? Didn't it always use to?
print(top_df)

     year month                                                url
0    2016   Jan  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
1    2016   Feb  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
2    2016   Mar  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
3    2016   Apr  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
4    2016   May  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
5    2016   Jun  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
6    2016   Jul  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
7    2016   Aug  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
8    2016   Sep  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
9    2016   Oct  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
10   2016   Nov  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
11   2015   Jan  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
12   2015   Feb  http://www.e-stat.go.jp/SG1/estat/ListE.do?lid...
13   2015   Mar  http://www.e-stat.go.jp/SG1/estat/ListE.do?li

In [7]:
def retrieve_month_excels(year, month, murl):
    mcols = ['year','month', 'excel_num', 'excel_description', 'excel_url']
    mdf = pd.DataFrame(columns=mcols)

#     year='2016'
#     month='Jan'
#     murl = df['url'][0]

    mr = re.get(murl)
    mdata = mr.text
    msoup = BeautifulSoup(mdata, "lxml")
    counter=0
    for elem in msoup.find_all('tr'):
        links = elem.find_all('a')
        if len(links) != 0:
            for link in links:
                if link.img.get('alt') == 'Excel File':
                    num = elem.td.string.strip()

                    excel_description = 'Not described'
                    tablerow=elem.find_all('td')
                    for cell in tablerow:
                        if cell.get('class')[0] == 'topborder':
                            excel_description = cell.string.strip()

                    excel_url = path + link.get('href').strip('.')

                    mrow = pd.DataFrame([[year, month, num, excel_description, excel_url]], columns=mcols)
                    mdf = mdf.append(mrow, ignore_index=True)
    return(mdf)
    # print(len(mdf))

In [14]:
%%time
cols = ['year','month', 'excel_num', 'excel_description', 'excel_url']
excelref_df = pd.DataFrame(columns = cols)
for index, row in top_df.iterrows():
    excelref_df = excelref_df.append(retrieve_month_excels(row['year'], row['month'], row['url']), ignore_index=True)
print(excelref_df)

     year month excel_num                                  excel_description  \
0    2016   Jan         1  Number of Intra-prefectural Migrants, In-migra...   
1    2016   Jan         2  Number of Inter-prefectural Migrants by Sex an...   
2    2016   Jan       3-1  Number of In-migrants from Other Prefectures b...   
3    2016   Jan       3-2  Number of Out-migrants to Other Prefectures by...   
4    2016   Jan       3-3  Number of Net-migration by Age (5-Year Age Gro...   
5    2016   Feb         1  Number of Intra-prefectural Migrants, In-migra...   
6    2016   Feb         2  Number of Inter-prefectural Migrants by Sex an...   
7    2016   Feb       3-1  Number of In-migrants from Other Prefectures b...   
8    2016   Feb       3-2  Number of Out-migrants to Other Prefectures by...   
9    2016   Feb       3-3  Number of Net-migration by Age (5-Year Age Gro...   
10   2016   Mar         1  Number of Intra-prefectural Migrants, In-migra...   
11   2016   Mar         2  Number of Int

In [41]:
exceldesccount_df = excelref_df.groupby(['excel_description']).agg(['count']).sort_values([('year', 'count')], ascending=False).ix[:,:1]
exceldesccount_df.columns=['count']
# exceldesccount_df.insert(0,'excel_description', exceldesccount_df.index)
exceldesccount_df.reset_index()

,excel_description,count
0,Number of In-migrants from Other Prefectures b...,55
1,Number of Out-migrants to Other Prefectures by...,55
2,Number of Net-migration by Age (5-Year Age Gro...,42
3,Number of Net-migration by Age (5-Year Age Gro...,41
4,Number of Inter-prefectural Migrants by Sex an...,28
5,"Number of Intra-prefectural Migrants, In-migra...",28
6,Number of In-migrants from Other Prefectures b...,28
7,Number of Out-migrants to Other Prefectures by...,28
8,Number of Inter-prefectural Migrants by Sex an...,28
9,Number of Inter-prefectural Migrants by Sex an...,27


In [43]:
print(excelref_df['excel_url'][4])

http://www.e-stat.go.jp/SG1/estat/XlsdlE.do?sinfid=000031369590
